<a href="https://colab.research.google.com/github/ChetHefton/4205_Group_Project/blob/main/loan_default_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd #data handling
import numpy as np #math and arrays

from sklearn.model_selection import train_test_split #split into train and validation
from sklearn.preprocessing import StandardScaler #normalizes
from sklearn.metrics import ( #import various metrics
    accuracy_score, confusion_matrix, roc_curve, roc_auc_score, classification_report
)

from sklearn.linear_model import LogisticRegression #baseline model
from sklearn.tree import DecisionTreeClassifier #use for comparison
#advance model 1 & 2 vvv
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt #visualize
import seaborn as sns #customization